In [5]:
import logging
from abc import ABC, abstractmethod 
from typing import Union
import numpy as np 
import pandas as pd
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split


class DataStrategy(ABC):
    """
    Abstract class for handling data
    """
    @abstractmethod
    def handle_data(self, data: pd.DataFrame) -> Union[pd.DataFrame, pd.Series]:
        pass

class DataPreProcessStrategy(DataStrategy):
    """
    Class for cleaning data
    """
    def handle_data(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Preprocess data
        """
        try:
            encoder = LabelEncoder()

            # Encoding 'sex' and 'smoker' columns
            data['sex'] = encoder.fit_transform(data['sex'])
            data['smoker'] = encoder.fit_transform(data['smoker'])

            # Dropping less correlated columns based on the correlation matrix
            data = data.drop(
                ['region', 'children', 'sex'],
                axis=1
            )

            return data
        except Exception as e:
            logging.error(f"Error in cleaning data: {e}")
            raise e
        
class DataDivideStrategy(DataStrategy):
    """
    Split the data into train and test sets
    """
    def handle_data(self, data: pd.DataFrame) -> Union[tuple, None]:
        """
        Split the data into training and test sets and return them.
        """
        try:
            y = data['charges']  # Target column
            X = data[['smoker', 'age', 'bmi']]  # Features
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            return X_train, X_test, y_train, y_test
        except Exception as e:
            logging.error(f"Error in splitting data: {e}")
            raise e


class DataCleaning:
    """
    Class to clean and split the data
    """
    def __init__(self, data: pd.DataFrame, strategy: DataStrategy):
        self.data = data
        self.strategy = strategy

    def handle_data(self) -> Union[pd.DataFrame, pd.Series]:
        """
        Clean the data
        """
        try:
            return self.strategy.handle_data(self.data)
        except Exception as e:  
            logging.error(f"Error in cleaning data: {e}")
            raise e

In [6]:
df_test = pd.DataFrame(data=[
                [19,"female",27.9,0,"yes","southwest",16884.924],
                [18,"male",33.77,1,"no","southeast",1725.5523],
                [28,"male",33,3,"no","southeast",4449.462]
            ],
            columns=["age","sex","bmi","children","smoker","region","charges"])

df_test

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523
2,28,male,33.00,3,no,southeast,4449.4620


In [7]:
cleaning_test = DataPreProcessStrategy()

In [8]:
cleaning_test.handle_data(df_test)

,age,bmi,smoker,charges
0,19,27.90,1,16884.9240
1,18,33.77,0,1725.5523
2,28,33.00,0,4449.4620
